# Re-map FLAVA checkpoint

Modifying FLAVA's components can cause existing model checkpoints to go out of sync with the updated architecture. This notebook shows how to load the existing checkpoint, re-map the old layers to the new layers, and save the new checkpoint.

To upload a new checkpoint, you must have access to the PyTorch AWS S3 account, and manually upload it from a local copy.

### Load original model

Load the existing checkpoint into the FLAVA class to see what the architecture currently is.

In [3]:
import torch
from torchmultimodal.models.flava.model import flava_model_for_classification, flava_model_for_pretraining

# flava_classification = flava_model_for_classification(num_classes=3)
flava_pretraining = flava_model_for_pretraining(pretrained_model_key='flava_full')

### Print summary

In [ ]:
flava_pretraining

### Mapping function

Replace this function with the code needed to map the old layer weights to the new layer weights.

In [4]:
import re

def map_state_dict(state_dict):
    mapped_state_dict = {}
    for param, val in state_dict.items():
        res = re.search('attention.attention', param)
        if res:
            idx = res.start()
            new_param = param[:idx] + param[idx+10:]
        else:
            new_param = param
        mapped_state_dict[new_param] = val
    return mapped_state_dict

### Load old state dict

In [5]:
# Load from url, replace this path if it changes
# old_model_url = 'https://download.pytorch.org/models/multimodal/flava/flava_model.pt'
# old_state_dict = torch.hub.load_state_dict_from_url(old_model_url)

# Or get from loaded model
old_state_dict = flava_pretraining.model.state_dict()

### Perform re-mapping

In [6]:
#new_state_dict = map_state_dict(old_state_dict)
new_state_dict = old_state_dict

### Save updated checkpoint

In [7]:
save_path = '/Users/rafiayub/flava_model.pt'
torch.save(new_state_dict, save_path)